In [ ]:
# !pip install llama-parse
# !pip install llama-index-vector-stores-chroma
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-llms-langchain
# !pip install llama-index-llms-gemini

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv()

In [ ]:
from llama_parse import LlamaParse  # pip install llama-parse
import nest_asyncio
nest_asyncio.apply()

parser = LlamaParse(
    api_key="llx-njIVCaPDqYOUfzRcFanrVBEYX6tuB8MXzBnCCGTZcxNs0HhY",  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown"  # "markdown" and "text" are available
)

# sync
documents = parser.load_data("386753.pdf")

In [52]:
documents

[Document(id_='2755abf2-9308-4671-8634-4c475a03871f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="|UPSTREAM ASSET FACTSHEET|January 2024|\n|---|---|\n|LIEGE|Canada|\n|Erik Wold|Rystad Energy|\n---\n|Contents|Page|\n|---|---|\n|Executive Summary|3|\n|Map|5|\n|Petroleum System|7|\n|Resources and production|8|\n|Cash flows and valuation|10|\n|Transactions|14|\n|Assumptions|15|\n|Glossary|16|\n\nErik Wold\n\nRystad Energy\n---\n## Executive Summary\n\nThe Liege project is located in the Western Canada Resources, which has a 100% working interest. Sedimentary basin in Canada. The project is expected to be approved in 2058 and start production in 2063. The project is expected to reach peak production in 2068 at 32,000 boepd. The primary field, Liege, CA will be developed with onshore. Production will be exported via onshore pipeline for liquids. Original recoverable resources are estimated at 421 million boe, with 100% 

In [53]:
documents[0].text

"|UPSTREAM ASSET FACTSHEET|January 2024|\n|---|---|\n|LIEGE|Canada|\n|Erik Wold|Rystad Energy|\n---\n|Contents|Page|\n|---|---|\n|Executive Summary|3|\n|Map|5|\n|Petroleum System|7|\n|Resources and production|8|\n|Cash flows and valuation|10|\n|Transactions|14|\n|Assumptions|15|\n|Glossary|16|\n\nErik Wold\n\nRystad Energy\n---\n## Executive Summary\n\nThe Liege project is located in the Western Canada Resources, which has a 100% working interest. Sedimentary basin in Canada. The project is expected to be approved in 2058 and start production in 2063. The project is expected to reach peak production in 2068 at 32,000 boepd. The primary field, Liege, CA will be developed with onshore. Production will be exported via onshore pipeline for liquids. Original recoverable resources are estimated at 421 million boe, with 100% liquids content. According to Rystad Energy’s base case scenario, the NPV for the field is estimated at $1.8 billion. The field is operated by Strathcona\n\n|Recoverable 

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("test2")

# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [ ]:
from llama_index.core import Settings
import os
from llama_index.llms.gemini import Gemini

GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = Gemini(api_key=GEMINI_API_KEY)
Settings.llm = llm

In [ ]:
from llama_index.core import StorageContext
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

# Query Data
query_engine = index.as_query_engine()

In [55]:
response = query_engine.query("first list the revenue of each year 2060 to 2075, and total of Economics Million USD revenue at 2060 to 2075?")
display((f"<b>{response}</b>"))

'<b>|Year|Revenue|\n|---|---|\n|2060|0|\n|2061|0|\n|2062|0|\n|2063|272|\n|2064|439|\n|2065|643|\n|2066|921|\n|2067|1,358|\n|2068|1,541|\n|2069|1,580|\n|2070|1,620|\n|2071|1,660|\n|2072|1,702|\n|2073|1,744|\n|2074|1,788|\n|2075|1,832|\n|Total|12,118|</b>'

In [56]:
response = query_engine.query("what is project revenue for Economics Million USD revenue at 2100?")
display((f"<b>{response}</b>"))

'<b>The provided context does not contain information about project revenue for Economics Million USD revenue at 2100.</b>'

In [ ]:
18+17+17+17+16+16+15+15+13+9+6+4+2